# Problem Statement
Monitoring and managing vehicle flow in real-time is critical for optimizing traffic management, ensuring road safety, and enhancing infrastructure planning. However, traditional methods rely on costly hardware installations (e.g., loop detectors, cameras with fixed thresholds), which can be inefficient and fail to adapt to dynamic environments.

### Practical Application
- **Smart Traffic Management System**:
This vehicle counter project provides an AI-powered solution to count and classify vehicles (e.g., cars, buses, trucks) in real-time using video feeds. It can be deployed for:

- **Traffic Flow Analysis:** Monitor and analyze vehicle volume on highways or intersections to reduce congestion.
- **Toll Booth Automation:** Count and classify vehicles for automated toll collection systems.
- **Accident Detection:** Track abnormal vehicle behavior (e.g., sudden stops) for emergency alerts.
- **Urban Planning:** Use traffic data to design better road networks and optimize signal timings.


By leveraging AI-based detection and tracking, this system offers a scalable and cost-effective alternative to traditional traffic monitoring methods.

### YOLOv8 vehicle counter

In [10]:
from ultralytics import YOLO
import cv2, time

def run_yolov8_tracking_with_counter(video_source, model_path):
    """
    Run vehicle detection, tracking, and counting using YOLOv8 with class names.
    Args:
        video_source: Path to the video file or webcam index (e.g., 0 for default webcam).
        model_path: Path to YOLOv8 model (e.g., "yolov8n.pt").
    """
    # Load YOLOv8 model
    model = YOLO(model_path)

    # Open the video source
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print('Error: Unable to open video source.')
        return

    # Vehicle counters and helper variables
    entered_vehicle_ids = []
    exited_vehicle_ids = []

    # Map of vehicle class IDs to names
    class_names = model.names
    vehicle_class_ids = [1, 2, 3, 5, 7]  # IDs for bicycle, car, motorcycle, bus, truck
    vehicle_entry_count = {class_names[cls]: 0 for cls in vehicle_class_ids}
    vehicle_exit_count = {class_names[cls]: 0 for cls in vehicle_class_ids}

    # Entry and exit lines
    entry_line = {'x1': 160, 'y1': 558, 'x2': 708, 'y2': 558}
    exit_line = {'x1': 1155, 'y1': 558, 'x2': 1718, 'y2': 558}
    offset = 20

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Exit the loop if no more frames

        start_time = time.time()  # Start timer for FPS calculation

        # Perform detection and tracking
        results = model.track(frame, persist=True, tracker="bytetrack.yaml")  # Use built-in ByteTrack tracker

        # Ensure results contain boxes
        if results and results[0].boxes is not None:
            for box in results[0].boxes:
                # Extract bounding box coordinates, class ID, confidence, and track ID
                x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy().flatten())  # Bounding box coordinates
                cls = int(box.cls.cpu().numpy().item())  # Class ID
                conf = float(box.conf.cpu().numpy().item())  # Confidence score
                track_id = int(box.id.cpu().numpy().item()) if box.id is not None else None  # Track ID (if available)

                # Get the class name
                class_name = class_names[cls]

                # Calculate the center of the bounding box
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                # Draw bounding box and label
                color = (0, 255, 0)  # Green color for vehicles
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                #label = f"ID: {track_id}, Class: {class_name}, Conf: {conf:.2f}"
                label = f"{class_name}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Count vehicles crossing entry line
                if entry_line['x1'] <= center_x <= entry_line['x2'] and entry_line['y1'] <= center_y <= entry_line['y1'] + offset:
                    if track_id not in entered_vehicle_ids and cls in vehicle_class_ids:
                        vehicle_entry_count[class_name] += 1
                        entered_vehicle_ids.append(track_id)

                # Count vehicles crossing exit line
                if exit_line['x1'] <= center_x <= exit_line['x2'] and exit_line['y1'] - offset <= center_y <= exit_line['y1']:
                    if track_id not in exited_vehicle_ids and cls in vehicle_class_ids:
                        vehicle_exit_count[class_name] += 1
                        exited_vehicle_ids.append(track_id)

        # Draw overlays for entry and exit counters
        y_pos = 100
        cv2.putText(frame, "EXIT", (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 250), 3)
        for idx, (cls_name, count) in enumerate(vehicle_entry_count.items()):
            y_pos = 150 + idx * 30
            cv2.putText(frame, f"{cls_name}: {count}", (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        y_pos = 100
        cv2.putText(frame, "ENTRY", (1710, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 250), 3)
        for idx, (cls_name, count) in enumerate(vehicle_exit_count.items()):
            y_pos = 150 + idx * 30
            cv2.putText(frame, f"{cls_name}: {count}", (1710, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Draw the counting lines
        cv2.line(frame, (entry_line['x1'], entry_line['y1']), (entry_line['x2'], entry_line['y2']), (0, 127, 255), 3)
        cv2.line(frame, (exit_line['x1'], exit_line['y1']), (exit_line['x2'], exit_line['y2']), (0, 127, 255), 3)

        # Display FPS
        fps = 1 / (time.time() - start_time)
        cv2.putText(frame, f"FPS: {fps:.2f}", (20, 52), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow('YOLOv8 Vehicle Tracking', cv2.resize(frame, (1280, 720)))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  # Exit when 'q' is pressed

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

    # Print final counts
    print("Vehicle Entry Counts:", vehicle_entry_count)
    print("Vehicle Exit Counts:", vehicle_exit_count)


In [17]:
run_yolov8_tracking_with_counter('videos/highway.mp4', 'model/yolov8s.pt')


0: 384x640 3 cars, 879.8ms
Speed: 7.0ms preprocess, 879.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 595.4ms
Speed: 11.0ms preprocess, 595.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 498.3ms
Speed: 6.0ms preprocess, 498.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 473.7ms
Speed: 11.0ms preprocess, 473.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 490.7ms
Speed: 7.0ms preprocess, 490.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 466.8ms
Speed: 11.0ms preprocess, 466.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 402.4ms
Speed: 7.0ms preprocess, 402.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 429.4ms
Speed: 7.0ms preprocess, 429.4ms inference, 2.0ms postprocess per image at shape (1, 3,

## Dynmaic vehicle counter

In [6]:
from ultralytics import YOLO
import cv2, time

def run_yolov8_generalized_counter(video_source, model_path, counting_lines):
    """
    Run vehicle detection, tracking, and counting using YOLOv8 with generalized counting lines.
    Args:
        video_source: Path to the video file or webcam index (e.g., 0 for default webcam).
        model_path: Path to YOLOv8 model (e.g., "yolov8n.pt").
        counting_lines: List of counting line configurations (e.g., entry/exit lines).
    """
    # Load YOLOv8 model
    model = YOLO(model_path)

    # Open the video source
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print('Error: Unable to open video source.')
        return

    # Get original video resolution
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Original video resolution: {original_width}x{original_height}")

    # YOLOv8 resolution
    yolo_width, yolo_height = 640, 384  # From YOLOv8 logs

    # Scale counting lines to YOLOv8 resolution
    scaled_lines = []
    for line in counting_lines:
        scaled_line = {
            "label": line["label"],
            "x1": int(line["x1"] * yolo_width / original_width),
            "y1": int(line["y1"] * yolo_height / original_height),
            "x2": int(line["x2"] * yolo_width / original_width),
            "y2": int(line["y2"] * yolo_height / original_height),
            "offset": int(line["offset"] * yolo_height / original_height),
        }
        scaled_lines.append(scaled_line)
    print(f"Scaled lines: {scaled_lines}")

    # Vehicle counters and helper variables
    class_names = model.names
    vehicle_class_ids = [1, 2, 3, 5, 7]  # IDs for bicycle, car, motorcycle, bus, truck
    counters = {line["label"]: {class_names[cls]: 0 for cls in vehicle_class_ids} for line in scaled_lines}
    tracked_ids = {line["label"]: [] for line in scaled_lines}  # Track IDs for each line

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Exit the loop if no more frames

        start_time = time.time()  # Start timer for FPS calculation

        # Resize frame to YOLOv8's input resolution
        resized_frame = cv2.resize(frame, (yolo_width, yolo_height))

        # Perform detection and tracking
        results = model.track(resized_frame, persist=True, tracker="bytetrack.yaml")  # Use built-in ByteTrack tracker

        if results and results[0].boxes is not None:
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy().flatten())  # Bounding box coordinates
                cls = int(box.cls.cpu().numpy().item())  # Class ID
                track_id = int(box.id.cpu().numpy().item()) if box.id is not None else None
                conf = box.conf.numpy().item()
                
                # Calculate the center of the bounding box
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                # Check crossing for each scaled line
                for line in scaled_lines:
                    label, lx1, ly1, lx2, ly2, offset = line.values()
                    if lx1 <= center_x <= lx2 and ly1 - offset <= center_y <= ly1 + offset:
                        if track_id not in tracked_ids[label] and cls in vehicle_class_ids:
                            counters[label][class_names[cls]] += 1
                            tracked_ids[label].append(track_id)

                # Draw bounding box
                color = (0, 255, 0)
                cv2.rectangle(resized_frame, (x1, y1), (x2, y2), color, 1)
                #label = f"ID: {track_id}, Class: {class_names[cls]}, Conf: {conf:.2f}"
                label = class_names[cls]
                cv2.putText(resized_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Draw scaled lines
        for idx, line in enumerate(scaled_lines):
            label, lx1, ly1, lx2, ly2, offset = line.values()

            # Draw the counting line
            cv2.line(resized_frame, (lx1, ly1), (lx2, ly2), (0, 127, 255), 3)

        # Draw vertical counters for each line
        x_start = 30  # Starting x position for the first column
        x_offset = 200  # Horizontal distance between columns
        y_start = 30  # Starting y position for rows
        y_offset = 30  # Vertical spacing between rows

        for col_idx, line in enumerate(scaled_lines):
            label = line["label"]
            x_pos = x_start + col_idx * x_offset  # Calculate column position for this line

            # Display the line label (e.g., "Entry1 Counts")
            cv2.putText(
                resized_frame, 
                f"{label} Counts:", 
                (x_pos, y_start), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5,  # Font scale
                (255, 255, 255),  # White color
                1  # Thickness
            )

            # Display each class count in a vertical list
            for row_idx, (cls_name, count) in enumerate(counters[label].items()):
                y_pos = y_start + (row_idx + 1) * y_offset  # Calculate row position
                cv2.putText(
                    resized_frame, 
                    f"{cls_name}: {count}", 
                    (x_pos, y_pos), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5,  # Font scale
                    (255, 255, 255),  # White color
                    1  # Thickness
                )

        # Display FPS
        fps = 1 / (time.time() - start_time)
        fps_text = f"FPS: {fps:.2f}"
        cv2.putText(resized_frame, fps_text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Resize for display only
        display_frame = cv2.resize(resized_frame, (original_width, original_height))  # Scale back for visualization
        cv2.imshow("YOLOv8 Vehicle Tracking", cv2.resize (display_frame, (960, 540)))
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    for label, counts in counters.items():
        print(f"{label} Counts:", counts)


### Testing our Dynamic counter
#### creating a function to get the coordnates (x,y)

In [ ]:
import cv2

def get_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        print(f"Selected point: ({x}, {y})")

cap = cv2.VideoCapture('./highway2.mp4')
count = 0
while count < 6:
    ret, frame = cap.read()
    if ret:
        cv2.imshow("Frame", cv2.resize (frame, (960, 540)))
    if count == 3:
        cv2.setMouseCallback("Frame", get_coordinates)  # Attach the callback
        cv2.waitKey(0)  # Wait for user to click
    count +=1
cap.release()
cv2.destroyAllWindows()


In [8]:
counting_lines = [
    {"label": "Entry1", "x1": 469, "y1": 397, "x2": 814, "y2": 449, "offset": 20},
    {"label": "Entry2", "x1": 1037, "y1": 424, "x2": 1431, "y2": 346, "offset": 20}
]

In [12]:
run_yolov8_generalized_counter("videos/highway3.mp4", "model/yolov8s.pt", counting_lines)

Original video resolution: 1920x1080
Scaled lines: [{'label': 'Entry1', 'x1': 156, 'y1': 141, 'x2': 271, 'y2': 159, 'offset': 7}, {'label': 'Entry2', 'x1': 345, 'y1': 150, 'x2': 477, 'y2': 123, 'offset': 7}]

0: 384x640 2 cars, 1 train, 864.5ms
Speed: 9.0ms preprocess, 864.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1196.3ms
Speed: 77.8ms preprocess, 1196.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 train, 750.1ms
Speed: 5.8ms preprocess, 750.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 train, 412.7ms
Speed: 8.0ms preprocess, 412.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 train, 426.9ms
Speed: 4.0ms preprocess, 426.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 512.2ms
Speed: 5.0ms preprocess, 512.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 64